In [ ]:
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [ ]:
!wget https://github.com/TobiasGlaubach/python-ml-turbofan/archive/master.zip

In [ ]:
!unzip master.zip


In [ ]:
!ls

In [ ]:
path='/home/spark/shared/python-ml-turbofan-master/CMAPSSData'
print(path)


In [ ]:
os.listdir(path)

In [ ]:
train_files= [filename for filename in os.listdir(path) if(filename.startswith('train') and filename.endswith('.txt'))]
                                                           

In [ ]:
train_files[0]

In [ ]:
test_files= [filename for filename in os.listdir(path) if(filename.startswith('test') and filename.endswith('.txt'))]

In [ ]:
print(train_files)
test_files

In [ ]:
!ls

In [ ]:
train_rdd=sc.textFile(path+'/'+train_files[0])

In [ ]:
train_rdd=train_rdd.map(lambda line:line.split(" "))

In [ ]:
train_rdd.count()

In [ ]:
test_rdd=sc.textFile(path+'/'+test_files[0])
test_rdd=test_rdd.map(lambda line:line.split(" "))

In [ ]:
test_rdd.count()

In [ ]:
header = ["id", "cycle", "setting1", "setting2", "setting3",
                    "s1", "s2", "s3", "s4", "s5", "s6", "s7", "s8",
                    "s9", "s10", "s11", "s12", "s13", "s14", "s15", 
                    "s16","s17", "s18", "s19", "s20", "s21"]

In [ ]:
train_df=train_rdd.toDF(header)

In [ ]:
train_df.collect()

In [ ]:
test_df=test_rdd.toDF(header)

In [ ]:
test_df.show()

In [ ]:
import pandas as pd
train_pandas_df=train_df.toPandas()

In [ ]:
train_pandas_df.head()

In [ ]:
test_pandas_df=test_df.toPandas()
test_pandas_df.head()

In [ ]:
train_pandas_df.isnull().sum()


In [ ]:
train_df.na.drop()

In [ ]:
test_df.na.drop()

In [ ]:
train_pandas_df.dropna()

In [ ]:
df = train_df.withColumn('id', train_df['id'].cast(IntegerType()))

In [ ]:
df.collect()

In [ ]:
def set_type(df,intlist=[],doublelist=[]):
    if len(intlist)>0:
        for col in intlist:
            df=df.withColumn(col,df[col].cast(IntegerType()))
                             
    if len(doublelist)>0:
        for col in doublelist:
            df=df.withColumn(col,df[col].cast(DoubleType()))
    return df                            
                             

In [ ]:
train_df.columns

In [ ]:
int_list = ["id", "cycle"]
double_list= list(set(train_df.columns)-set(int_list) )
double_list

In [ ]:
train_df=set_type(train_df,int_list,double_list)
test_df=set_type(test_df,int_list,double_list)

In [ ]:
train_df.columns

In [ ]:
test_df.schema

In [ ]:
train_df.schema

In [ ]:
print ("For train data:")
train_df.groupBy('id').count().sort(F.col("count")).show(1)


In [ ]:
train_df.groupBy('id').count().sort(F.col("count")).show(1)

In [ ]:
train_df.groupBy('id').count().sort(F.col("count").desc()).show(1)

In [ ]:
test_df.groupBy('id').count().show()

In [ ]:
train_df.toPandas().count()

# 2020-02-04 FEATURE SCALING

In [ ]:
rul=train_df.groupBy('id').agg({"cycle":"max"}).sort(("id"))

In [ ]:
type(rul)

In [ ]:
rul=rul.toDF('id','max')

In [ ]:
rul.show()

In [ ]:
temp_df=train_df.join(rul,on=['id'],how='left').toPandas()

In [ ]:
temp_df.sort_values(by=["id"])

In [ ]:
temp_df['RUL']=temp_df['max']-temp_df['cycle']

In [ ]:
temp_df.sort_values(by=["id","cycle"])

In [ ]:
def add_rul_labelling(df,id,time):
    rul=df.groupBy(id).agg({time:"max"}).sort('id')
    rul=rul.toDF('id','max')
    df=df.join(rul,on=['id'],how='left')
    df=df.withColumn("rul",df["max"]-df[time])
    return df

In [ ]:
df=add_rul_labelling(train_df,"id","cycle")

In [ ]:
df.toPandas()

In [ ]:
def add_window_labelling(df,w1,w2):
    df=df.withColumn('label1',F.when(df.rul<w1,1).otherwise(0))
    df=df.withColumn('label2',df.label1)
    df=df.withColumn('label2',F.when(df.rul<w2,2).otherwise(df.label1))
    return df

In [ ]:
train_df.collect()

In [ ]:
df=add_window_labelling(df,30,15)

In [ ]:
df.toPandas()

# 2020/02/05


Convert ground truth file (RUL**) to spark data frame(df_gt)
add uniquie id to new data frame based on the order of valuse
create another RUL data farme from the test file (rul)
concatenate df_gt and rul dataframes based on id 
derive total RUL from these 2 DFs


In [ ]:
path

In [ ]:
os.listdir(path)

In [ ]:
GT_FILE_TYPE="RUL"
gt_files=[filename for filename  in os.listdir(path) if filename.startswith(GT_FILE_TYPE) ]
gt_files

In [ ]:
rdd_gt=sc.textFile(path+'/'+gt_files[0])
rdd_gt=rdd_gt.map(lambda line : line.split(" "))

In [ ]:
rdd_gt.count()

In [ ]:
header=["rul"]
df_gt=rdd_gt.toDF(header)

In [ ]:
df_gt=df_gt.drop("_2")

In [ ]:
df_gt=df_gt.withColumn("rul",df_gt["rul"].cast(IntegerType()))

In [ ]:
df_gt=df_gt.drop("_2")

In [ ]:
w=Window.orderBy(df_gt["rul"])

In [ ]:
df_gt=df_gt.withColumn("id",F.row_number().over(w))

In [ ]:
rul=test_df.groupBy("id").agg({"cycle":"max"}).sort("id")

In [ ]:
rul=rul.toDF("id","max")

In [ ]:
rul.show()

In [ ]:
df_gt.show()

In [ ]:
df_gt=df_gt.join(rul,on=["id"],how='left')


In [ ]:
df_gt=df_gt.orderBy("id")

In [ ]:
df_gt=df_gt.withColumn("total_rul",df_gt["rul"]+df_gt["max"])

In [ ]:
df_gt.show()

# 2020/02/06

In [ ]:
df_gt=df_gt.drop("rul").drop("max")

In [86]:
test_df=test_df.join(df_gt,on="id",how="left")
test_df.toPandas()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s15,s16,s17,s18,s19,s20,s21,_27,_28,total_rul
0,31,1,-0.0019,-0.0004,100.0,518.67,642.75,1582.03,1392.43,14.62,...,8.4028,0.03,394.0,2388.0,100.0,38.83,23.4048,None,None,246
1,31,2,-0.0016,-0.0001,100.0,518.67,642.02,1586.39,1398.94,14.62,...,8.4148,0.03,391.0,2388.0,100.0,39.03,23.2768,None,None,246
2,31,3,-0.0043,0.0004,100.0,518.67,642.36,1590.20,1405.66,14.62,...,8.4092,0.03,391.0,2388.0,100.0,38.99,23.3628,None,None,246
3,31,4,-0.0011,-0.0001,100.0,518.67,642.18,1582.10,1395.35,14.62,...,8.4430,0.03,392.0,2388.0,100.0,38.96,23.3472,None,None,246
4,31,5,-0.0039,0.0005,100.0,518.67,642.23,1582.58,1401.53,14.62,...,8.4152,0.03,393.0,2388.0,100.0,38.92,23.4112,None,None,246
5,31,6,-0.0025,-0.0003,100.0,518.67,642.30,1581.37,1393.71,14.62,...,8.3950,0.03,391.0,2388.0,100.0,39.03,23.4342,None,None,246
6,31,7,0.0007,0.0004,100.0,518.67,641.61,1588.16,1404.37,14.62,...,8.3915,0.03,393.0,2388.0,100.0,38.99,23.4166,None,None,246
7,31,8,0.0032,0.0005,100.0,518.67,642.21,1587.75,1402.96,14.62,...,8.3934,0.03,391.0,2388.0,100.0,38.82,23.4539,None,None,246
8,31,9,0.0034,0.0002,100.0,518.67,642.10,1589.82,1396.67,14.62,...,8.4147,0.03,391.0,2388.0,100.0,39.03,23.4222,None,None,246
9,31,10,-0.0041,-0.0001,100.0,518.67,642.06,1585.22,1401.20,14.62,...,8.4347,0.03,392.0,2388.0,100.0,38.93,23.3936,None,None,246


In [89]:
test_df=test_df.withColumn("rul",test_df["total_rul"]-test_df["cycle"])

In [90]:
test_df.toPandas()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s16,s17,s18,s19,s20,s21,_27,_28,total_rul,rul
0,31,1,-0.0019,-0.0004,100.0,518.67,642.75,1582.03,1392.43,14.62,...,0.03,394.0,2388.0,100.0,38.83,23.4048,None,None,246,245
1,31,2,-0.0016,-0.0001,100.0,518.67,642.02,1586.39,1398.94,14.62,...,0.03,391.0,2388.0,100.0,39.03,23.2768,None,None,246,244
2,31,3,-0.0043,0.0004,100.0,518.67,642.36,1590.20,1405.66,14.62,...,0.03,391.0,2388.0,100.0,38.99,23.3628,None,None,246,243
3,31,4,-0.0011,-0.0001,100.0,518.67,642.18,1582.10,1395.35,14.62,...,0.03,392.0,2388.0,100.0,38.96,23.3472,None,None,246,242
4,31,5,-0.0039,0.0005,100.0,518.67,642.23,1582.58,1401.53,14.62,...,0.03,393.0,2388.0,100.0,38.92,23.4112,None,None,246,241
5,31,6,-0.0025,-0.0003,100.0,518.67,642.30,1581.37,1393.71,14.62,...,0.03,391.0,2388.0,100.0,39.03,23.4342,None,None,246,240
6,31,7,0.0007,0.0004,100.0,518.67,641.61,1588.16,1404.37,14.62,...,0.03,393.0,2388.0,100.0,38.99,23.4166,None,None,246,239
7,31,8,0.0032,0.0005,100.0,518.67,642.21,1587.75,1402.96,14.62,...,0.03,391.0,2388.0,100.0,38.82,23.4539,None,None,246,238
8,31,9,0.0034,0.0002,100.0,518.67,642.10,1589.82,1396.67,14.62,...,0.03,391.0,2388.0,100.0,39.03,23.4222,None,None,246,237
9,31,10,-0.0041,-0.0001,100.0,518.67,642.06,1585.22,1401.20,14.62,...,0.03,392.0,2388.0,100.0,38.93,23.3936,None,None,246,236


In [91]:
test_df=test_df.drop("total_rul")

In [ ]:
test_df.orderBy("id").show()